# PM2.5 prediction

### Import libraries

In [1]:
import math
import os

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
from scipy.stats import stats
seed(1)
from fs import methods as m
from fs import model as ml
import geopandas as gpd
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score


### Data Import

In [ ]:
"""
1-->Pearson
2-->Spearmanr
3-->Kendall
4-->Fisher
5-->Random Forest
6-->Average
"""
FOLDER='6'
RESOLUTION = '0_1'
TARGET = 'pm25_st'
KNN = True
knn_value=10
NUMBER_OF_COVARIATES = 12


geopackages = os.listdir('assets/grids_'+RESOLUTION)


results1 = pd.DataFrame(columns=['MAE', 'MSE', 'R2'])
results2 = pd.DataFrame(columns=['MAE', 'MSE', 'R2'])



for index, grid in enumerate(geopackages):
    data = gpd.read_file('assets/grids_'+RESOLUTION+'/'+ grid)

    if KNN:
        data = m.process_data(data, knn_value)

    coll = pd.DataFrame()
    coll['pm25_st'] = data['pm25_st']
    coll['pm25_cams'] = data['pm25_cams']
    coll = coll.dropna(axis=0)

    #print('Mean Absolute Error: ',mean_absolute_error(coll['pm25_cams'], coll['pm25_st'], ))
    #print('Mean Squared Error: ',mean_squared_error(coll['pm25_cams'], coll['pm25_st']))
    #print('R2 score: ',r2_score(coll['pm25_cams'], coll['pm25_st']))

    data = data[~data[TARGET].isnull()]
    data = data.dropna(axis=1)

    data.pop('geometry')

    labels = pd.read_csv('assets/features_'+RESOLUTION+'/'+FOLDER+ '/'+grid[:-5]+'.csv')['Features']
    #labels = pd.read_csv('fs.csv')['Features']

    #read variables which are not null
    score_results = pd.DataFrame()

    #Store dataset in x and y variables
    X = pd.DataFrame(data=data, columns=labels ).dropna(axis = 1)
    X = X.iloc[:, :NUMBER_OF_COVARIATES].to_numpy()

    Y = pd.DataFrame(data=data, columns=[TARGET])
    Y = Y.values.ravel()


    #NUMBER_OF_COVARIATES = X.shape[1]
    y1 = np.array(Y)
    skf = KFold(n_splits=6, shuffle = True)
    mae_list1 = []
    mse_list1 = []
    r2_list1 = []
    mae_list2 = []
    mse_list2 = []
    r2_list2 = []
    print('---------'+ grid +'---------')


    i = 1
    for train_index, test_index in skf.split(X):
        print("Iteration n°:  ", i)
        i = i + 1
        model = None
        model = Sequential()
        model.add(Dense(NUMBER_OF_COVARIATES, input_dim=NUMBER_OF_COVARIATES, kernel_initializer='normal', activation='relu'))
        model.add(Dense(NUMBER_OF_COVARIATES, activation='relu'))
        model.add(Dense(1, activation='linear'))
        model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

        model.summary()
        X_train = X[train_index]
        X_val = X[test_index]
        y_train, y_val = y1[train_index], y1[test_index]




        y_train=np.reshape(y_train, (-1,1))
        y_val=np.reshape(y_val, (-1,1))

        scaler_x = MinMaxScaler()
        scaler_y = MinMaxScaler()

        scaler_x.fit(X_train)
        xtrain_scale=scaler_x.transform(X_train)
        scaler_x.fit(X_val)
        xval_scale=scaler_x.transform(X_val)

        scaler_y.fit(y_train)
        ytrain_scale=scaler_y.transform(y_train)
        scaler_y.fit(y_val)
        yval_scale=scaler_y.transform(y_val)




        history=model.fit(xtrain_scale, ytrain_scale, epochs=50, batch_size=1, verbose=0) #, validation_split=0.30)
        predictions = model.predict(xval_scale)

        predictions = scaler_y.inverse_transform(predictions)

        mae = mean_absolute_error(y_val, predictions)
        mse = mean_squared_error(y_val, predictions)
        r2 = r2_score(y_val, predictions)

        #print('Mean Absolute Error: ',mae)
        #print('Mean Squared Error: ',mse)
        #print('R2 score: ',r2)
        mae_list1.append(mae)
        mse_list1.append(mse)
        r2_list1.append(r2)

        data_new = gpd.read_file('assets/grids_'+RESOLUTION+'/'+ grid)
        data_new = data_new[~data_new[TARGET].isnull()]
        pm25_obs = data_new['pm25_st']
        X_new = pd.DataFrame(data=data_new, columns=labels).dropna(axis=1)
        X_new = X_new.iloc[:, :NUMBER_OF_COVARIATES]

        scaler_x = MinMaxScaler()
        scaler_x.fit(X_new)
        X_new_scale=scaler_x.transform(X_new)




        predictions_pm25 = model.predict(X_new_scale,  batch_size=1)
        predictions_pm25 = pd.Series(np.array(scaler_y.inverse_transform(predictions_pm25)).reshape((-1,)))
        e = pd.DataFrame()

        e['predictions_pm25'] = list(predictions_pm25)
        e['pm25_obs'] = list(pm25_obs)
        e = e.dropna(axis=0)

        mae = mean_absolute_error(e['pm25_obs'], e['predictions_pm25'])
        mse = mean_squared_error(e['pm25_obs'], e['predictions_pm25'])
        r2 = r2_score(e['pm25_obs'], e['predictions_pm25'])
        #print('Mean Absolute Error: ',mae)
        #print('Mean Squared Error: ',mse)
        #print('R2 score: ',r2)
        mae_list2.append(mae)
        mse_list2.append(mse)
        r2_list2.append(r2)

    avg_mae1 = np.mean(mae_list1)
    avg_mse1 = np.mean(mse_list1)
    avg_r21 = np.mean(r2_list1)

    avg_mae2 = np.mean(mae_list2)
    avg_mse2 = np.mean(mse_list2)
    avg_r22 = np.mean(r2_list2)

    print('---------VALIDATION WITH ARPA (TEST_SET)---------')
    print('Mean Absolute Error: ',avg_mae1)
    print('Mean Squared Error: ',avg_mse1)
    print('R2 score: ',avg_r21)

    print('---------VALIDATION WITH ARPA (ALL)---------')
    print('Mean Absolute Error: ',avg_mae2)
    print('Mean Squared Error: ',avg_mse2)
    print('R2 score: ',avg_r22)
    mae_list1 = []
    mse_list1 = []
    r2_list1 = []
    mae_list2 = []
    mse_list2 = []
    r2_list2 = []

    results1.loc[index]= [round(avg_mae1,3), round(avg_mse1, 3), round(avg_r21, 3)]
    results1.rename(index={index: grid}, inplace=True)

    results2.loc[index]= [round(avg_mae2,3), round(avg_mse2, 3), round(avg_r22, 3)]
    results2.rename(index={index: grid}, inplace=True)

results1 = results1.T
results2 = results2.T

results1.to_excel('assets/test/keras1'+RESOLUTION+'.xlsx')
results2.to_excel('assets/test/keras2'+RESOLUTION+'.xlsx')


---------grid_0_1_0418_0425_2021.gpkg---------
Iteration n°:   1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                156       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 325
Trainable params: 325
Non-trainable params: 0
_________________________________________________________________


2022-05-29 19:35:30.914628: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-29 19:35:30.915485: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Iteration n°:   2
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_4 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 13        
Total params: 325
Trainable params: 325
Non-trainable params: 0
_________________________________________________________________
Iteration n°:   3
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_7 (Dense)              (None, 12)                156      